ESTE NOTEBOOK SE UTILIZARÁ PARA LA GENERACION DEL MODEL DE RECOMENDACIÓN

In [19]:
import pandas as pd
import numpy as np
import os
import scipy as sp
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
ruta_model = os.path.join('./modelo.parquet')
table = pq.read_table(ruta_model)
modelo = table.to_pandas()
print(modelo.head())

       id                 app_name        genres
0  761140      Lost Summoner Kitty        Action
1  643980                Ironbound  Free to Play
2  670290  Real Pool 3D - Poolians        Casual
3  767400                  弹炸人2222        Action
4  772540    Battle Royale Trainer        Action


In [21]:
modelo = pd.pivot_table(modelo, index=['id', 'app_name'], aggfunc='sum').reset_index()

In [22]:
modelo = pd.get_dummies(modelo, columns=['genres'])

In [23]:
modelo = pd.pivot_table(modelo, index=['id', 'app_name'], aggfunc='sum').reset_index()

In [24]:
modelo

,id,app_name,genres_Accounting,genres_Action,genres_Adventure,genres_Animation &amp; Modeling,genres_Arcade,genres_Audio Production,genres_Beat 'em up,genres_Casual,...,genres_Software Training,genres_Soundtrack,genres_Sports,genres_Strategy,genres_Twin Stick Shooter,genres_Utilities,genres_Vacio,genres_Video Production,genres_Web Publishing,genres_Zombies
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28702,2028055,Tom Clancy's Ghost Recon Future Soldier - Seas...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28703,2028056,Worms Revolution Season Pass,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
28704,2028062,Call of Duty®: Black Ops II Season Pass,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28705,2028103,Assassin’s Creed® III Season Pass,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
aventura = modelo[modelo['genres_Adventure'] == 1]
print(aventura)

           id                                    app_name  genres_Accounting  \
96       3260  Safecracker: The Ultimate Puzzle Adventure                  0   
173      6010    Indiana Jones® and the Fate of Atlantis™                  0   
176      6040                                    The Dig®                  0   
185      6300              Dreamfall: The Longest Journey                  0   
210      7210                   Runaway, A Road Adventure                  0   
...       ...                                         ...                ...   
28674  774431     Casino Slot Machines - The Supreme King                  0   
28679  775460              The Cryptkeepers of Hallowford                  0   
28684  776420           Casino Slot Machines - The Spring                  0   
28691  901399            Sam & Max: The Devil’s Playhouse                  0   
28696  901663                       Sam & Max: Season Two                  0   

       genres_Action  genres_Adventure 

In [26]:
audio = modelo[modelo['genres_Audio Production'] == 1]
print(audio)

           id                                          app_name  \
2252   207340               Sound Forge Mac 2.0 - Steam Powered   
2892   218760                            Mixcraft 8 Home Studio   
3222   224380                  Virtual DJ - Broadcaster Edition   
3342   228180        Action! - Gameplay Recording and Streaming   
3735   241660                                        Ohm Studio   
...       ...                                               ...   
26179  697410                          Rytmik Cloud Expansion 2   
27314  725610                                      foreverloops   
27813  741540                      Rytmik Studio Supporter Pack   
27942  746390  PlayClaw 6 - Game Recording, Streaming, Overlays   
27951  746700                    Zoom Player 14 : Steam Edition   

       genres_Accounting  genres_Action  genres_Adventure  \
2252                   0              0                 0   
2892                   0              0                 0   
3222        

In [27]:
modelo.iloc[:,3:] = modelo.iloc[:,3:].astype('float64')


In [28]:
similarity = cosine_similarity(modelo.iloc[:,3:])

In [29]:
similarity

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [30]:
def game_recommendation(id):
    id = int(id)
    
    if id not in modelo['id'].values:
        return "El juego con el ID especificado no existe en la base de datos."
    
    # Obtener el índice del juego seleccionado
    indice_juego = modelo[modelo['id'] == id].index[0]
    
    # Imprimir el nombre del juego y el género
    nombre_juego = modelo.at[indice_juego, 'app_name']
    print(f"Juego seleccionado: {nombre_juego}")
    
    # Obtener los géneros del juego seleccionado
    generos_juego = [columna for columna in modelo.columns[3:] if modelo.at[indice_juego, columna] == 1]
    print(f"Géneros: {', '.join(generos_juego)}")
    
    # Calculamos la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = similarity[indice_juego]
    
    # Calculamos los índices de los juegos más similares (excluyendo el juego de entrada)
    similarity_games = similarity_scores.argsort()[::-1][1:6]
    
    # Obtenemos los nombres de los juegos 5 recomendados
    recommend = modelo.iloc[similarity_games]['app_name'].values.tolist()
    
    return recommend



In [31]:
game_recommendation(10) #Prueba Juegos de Acción

Juego seleccionado: Counter-Strike
Géneros: genres_Action


['SJSM - Karamari Hospital',
 'Magicka 2: Ice, Death and Fury',
 'Gun Wings - Original Soundtrack',
 "Uriel's Chasm 2: את",
 'Dead Realm - Soundtrack']

In [32]:
game_recommendation(3260) #Prueba Juegos de Aventura

Juego seleccionado: Safecracker: The Ultimate Puzzle Adventure
Géneros: genres_Adventure


['Creatures Such as We',
 'Out There: Ω Edition - Soundtrack',
 'Eschalon: Book III',
 'Violett Remastered',
 'Revenge of the Spirit: Rite of Resurrection']

In [33]:
game_recommendation(207340) #Prueba Juegos de Audio

Juego seleccionado: Sound Forge Mac 2.0 - Steam Powered
Géneros: genres_Audio Production


['Ohmicide',
 'Rytmik Lite Chiptune Synthesizer',
 'Rytmik Cloud Expansion 2',
 'Liquid Rhythm Randomizer',
 'MAGIX Music Maker Hip Hop Edition 6']

In [34]:
modelo.to_csv('./Data/modelo1.csv', index=False, encoding='utf-8')

In [35]:
def csv_a_parquet(ruta_csv, ruta_parquet):
    df = pd.read_csv(ruta_csv)
    table = pa.Table.from_pandas(df)
    pq.write_table(table, ruta_parquet)

In [36]:
csv_a_parquet('./Data/modelo1.csv', 'modelo1.parquet')